In [ ]:
from heapq import heapify, heapreplace#, heappop, heappush
import pandas as pd
import networkx as nx
import psycopg2 as pg
import json

In [ ]:
def reBuildGraph(G, edgesHeap, firstSplit):
    for item in edgesHeap:
        (heapValue, u, v, idedge, lengthOriginal, utilityValue, numSplit) = item
        #The number of segments the edge must be split into is 1 less the value stored in the heap
        numSplit = numSplit - 1
        if numSplit >= firstSplit:
            lengthSplitted = lengthOriginal/numSplit
            vertexStart = u

            G.remove_edge(u, v, key=idedge)
            for i in range(numSplit - 1):
                vertexEnd = idedge + '_' + str(i + 1)
                G.add_edge(vertexStart, vertexEnd, key=vertexEnd, idedge=vertexEnd, length=lengthSplitted, utilityvalue=utilityValue)
                vertexStart = vertexEnd
            keyLast = idedge + '_' + str(numSplit)
            G.add_edge(vertexStart, v, key=keyLast, idedge=keyLast, length=lengthSplitted, utilityvalue=utilityValue)

    return G

def loadMultiDiGraph():
    params = {'host':'localhost', 'port':'5432', 'database':'afterqualifying', 'user':'cristiano', 'password':'cristiano'}
    conn = pg.connect(**params)

    sqlQuery = '''	select	EDGE.IDVERTEXORIG_FK,
                            EDGE.IDVERTEXDEST_FK,
                            EDGE.IDEDGE,
                            EDGE.LENGTH,
                            EDGE.UTILITYVALUE
                    from	STREETSEGMENT as EDGE--, MUNICIPALITY
                    --where   MUNICIPALITY.DESCRIPTION = 'São Caetano do Sul' and
                    --        ST_Intersects(MUNICIPALITY.GEOM, EDGE.GEOM) '''
    dataFrameEdges = pd.read_sql_query(sqlQuery, conn)
    conn.close()

    G = nx.MultiDiGraph()
    for row in dataFrameEdges.itertuples():
        dictRow = row._asdict()
        
        G.add_edge(dictRow['idvertexorig_fk'], dictRow['idvertexdest_fk'], key=str(dictRow['idedge']),
                    u=dictRow['idvertexorig_fk'], v=dictRow['idvertexdest_fk'],
                    idedge=str(dictRow['idedge']), length=dictRow['length'], utilityvalue=dictRow['utilityvalue'])

    return G

def loadMultiGraphEdgesSplit(nIterations=9, maxDistance=None):
    #It must be a MultiDiGraph because besides it has multiple edges between the same nodes, networkx does not assure the order of edges.
    #Using a directed graph, the start node of an edge will always be the start node, avoiding errors in the reBuildGraph function.
    G = loadMultiDiGraph()

    if nIterations > 0:
        firstSplit = 2
        #The value must be negative because the data structure is a min heap
        edgesHeap = [(-1*data['length'], data['u'], data['v'], data['idedge'], data['length'], data['utilityvalue'], firstSplit) for u, v, data in G.edges(data=True)]
        heapify(edgesHeap)
    
        for i in range(round(len(edgesHeap) * nIterations)):
            #The value must be multiplied by -1 because the data structure is a min heap
            if maxDistance != None and -1 * edgesHeap[0][0] <= maxDistance:
                break
            
            #(heapValue, u, v, idedge, lengthOriginal, utilityValue, numSplit) = heappop(edgesHeap)
            (heapValue, u, v, idedge, lengthOriginal, utilityValue, numSplit) = edgesHeap[0]
            #The value must be negative because the data structure is a min heap
            heapValue = -1 * lengthOriginal/numSplit
            #The numSplit is prepared for the next time the edge may be splitted (numsplit + 1)
            heapreplace(edgesHeap, (heapValue, u, v, idedge, lengthOriginal, utilityValue, numSplit + 1))

        G = reBuildGraph(G, edgesHeap, firstSplit)

    return G.to_undirected()

G = loadMultiGraphEdgesSplit(nIterations=0)
distanceCutOff = 210
allPairs = dict(nx.all_pairs_dijkstra_path_length(G, cutoff=distanceCutOff, weight='length'))
edgesLength = {}
for start, end, ddict in G.edges(data=True):
    edgesLength[ddict['idedge']] = ddict['length']

In [ ]:
def loadSSMS():
    edgesWithStation = set()
    edgeIdStations = set()
    #fileName = './SSMS/1.json'
    fileName = './SSMS_1_Thread/finalResult.sol'

    if fileName.endswith('.json'):
        with open(fileName) as jsonFile:
            optimalSolution = json.load(jsonFile)

            for variable in optimalSolution['Vars']:
                if not variable['VTag'][0].startswith('pos'):
                    stringSplit = variable['VTag'][0].split('-')
                    edgesWithStation.add(stringSplit[1] + '-' + stringSplit[0] + '-' + stringSplit[2])
                    edgeIdStations.add(stringSplit[1])
                    
    elif fileName.endswith('.sol'):
        with open(fileName) as solFile:
            for line in solFile:
                if not line.startswith('pos') and not line.startswith('#'):
                    edgeId, value = line.split()
                    if round(float(value)) == 1:
                        stringSplit = edgeId.split('-')
                        edgesWithStation.add(stringSplit[1] + '-' + stringSplit[0] + '-' + stringSplit[2])
                        edgeIdStations.add(stringSplit[1])

    return edgesWithStation, edgeIdStations

def loadSASS():
    edgesWithStation = set()
    edgeIdStations = set()
    fileName = './SASS_Antigo/8/1.json'

    with open(fileName) as jsonFile:
        optimalSolution = json.load(jsonFile)

        for variable in optimalSolution['Vars']:
            edgesWithStation.add(variable['VTag'][0])
            edgeIdStations.add(variable['VTag'][0].split('-')[0])
    
    return edgesWithStation, edgeIdStations

def printErrorDistances(visitedVertices, allPairs, u, edgeId, edgeIdStations):
    if not u in visitedVertices:
        visitedVertices.add(u)

        if u in allPairs:
            for vertex, distance in allPairs[u].items():
                for start, end, ddict in G.edges(vertex, data=True):
                    if edgeId != ddict['idedge'] and ddict['idedge'] in edgeIdStations:
                        print("ERRO: ", edgeId, ddict['idedge'], distance, ddict['length'], edgesLength[edgeId])

edgesWithStation, edgeIdStations = loadSSMS()
#edgesWithStation, edgeIdStations = loadSASS()
visitedVertices = set()
for station in edgesWithStation:
    edgeSplit = station.split('-')
    edgeId = edgeSplit[0]
    u = int(edgeSplit[1]) if '_' not in edgeSplit[1] else edgeSplit[1]
    v = int(edgeSplit[2]) if '_' not in edgeSplit[2] else edgeSplit[2]

    printErrorDistances(visitedVertices, allPairs, u, edgeId, edgeIdStations)
    printErrorDistances(visitedVertices, allPairs, v, edgeId, edgeIdStations)